# Main Dataset clean

**Abstract:**
One-sentence description

**Description:**
In the following cell, I...


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

import sys
import os

sys.path.append("..")
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# plt.style.use('ggplot')
plt.rcParams['axes.facecolor'] = 'white'

In [2]:
from config import ROOT, SCHEME_87, PROVINCES_CHN2ENG

## Title

**Abstract:**
One-sentence description

**Description:**
In the following cell, I...


In [3]:
# Introduction of the original datasets.
readme = pd.read_excel(os.path.join(ROOT, "data/source/Zhou et al_2020_PNAS_dataset.xlsx"), sheet_name='Readme')

# driver factors to water use changes
driver_data = pd.read_excel(os.path.join(ROOT, "data/source/Zhou et al_2020_PNAS_dataset.xlsx"), sheet_name='D3')

# 展示上述数据的数据介绍
readme_drivers = readme.loc[39:47, ['Variables', 'Units', 'Definition']]
readme_drivers

,Variables,Units,Definition
39,PIRR,mm,Potential irrigation requirements
40,AIRR,mm,Freshwater availability allocated to irrigatio...
41,WCI,%,The ratio between the area equipped for water-...
42,IRR WUI,mm,See definiton above
43,revised IRR WUI,mm,IRR WUI revised by crop mix at 1975 to remove ...
44,Ratio of industrial water recycling,%,The ratio of recycled water to total industria...
45,Ratio of industrial water evaporated,%,The ratio of evaporated water to total industr...
46,IND WUI,m3 Yuan-1,See definiton above
47,revised IND WUI,m3 Yuan-1,IND WUI revised by industrial structure at 197...


**Output digestion:**

---
**Analysis:**
可以看到，用水驱动主要有以下几个变量：
- Year，年份，这些数据的年份都是从1975开始，到2013年
- PIRR，潜在的灌溉用水需求，根据文章里说，是由全球水文模型跑出来的，因此有不小的不确定性。
- AIRR，可供灌溉的淡水量，也是全球水文模型跑出来的
- WCI，灌溉装备了节水面积的灌区占灌区总面积的比例，比如滴灌、管灌、渠道硬化
- Ratio of industrial water recycling，工业用水是可以循环使用的，这是再利用的比例
- Ratio of industrial water evaporated，这才算是工业真正的耗水，也就是工业用水中的多少是被消耗掉的

对于该数据，有以下几个基本认识：

- 最小的灌溉需求都有315mm， 但灌溉可供水量是有差异的，因此有的地方可能水赤字，这个赤字有可能从地下水补给，对分析带来较大的影响。这些模型跑出来的数据对定量分析帮助有限。
- WUI直接由总用水量计算出，因此根本不能使用。
- 可以使用农业和工业的节水设施变量参与预测：节水灌溉比例、工业再利用率、工业蒸发率（耗水）

## Water Uses Values

**Abstract:**
One-sentence description

**Description:**
In the following cell, I...


In [4]:
# water use values data and some economic factors
values_data = pd.read_excel(os.path.join(ROOT, "data/source/Zhou et al_2020_PNAS_dataset.xlsx"), sheet_name='D1')
values_data = values_data.dropna(axis=1, how='all')

# 清洗数据的头部，让两行头部变一行
change_name_dic = {}
last_item = "None"
for col in values_data:
    second_row = values_data.loc[0, col]
    if "Unnamed" in col:
        change_name_dic[col] = last_item + ": " + second_row
    else:
        if type(second_row) is str:
            change_name_dic[col] = col + ": " + second_row
        last_item = col

values_data.rename(change_name_dic, axis=1, inplace=True)
values_data = values_data.drop(0)

# 重命名表头，取消两边的空格
values_data.rename({col: col.strip() for col in values_data}, axis=1, inplace=True)

# 更改正确的数据类型
values_data = values_data.astype(float, errors='ignore')
values_data['Year'] = values_data['Year'].astype(int)
values_data.iloc[:, 2:] = values_data.iloc[:, 2:].astype(float)

# 整合省份数据
perfectures = pd.read_csv(os.path.join(ROOT, 'data/source/perfectures.csv')).drop(['FID', 'Shaoefile_'], axis=1)
values_data = pd.merge(left=values_data, right=perfectures, left_on='City_ID', right_on='Perfecture', right_index=False)

values_data.head()

,City_ID,Year,IRR,Irrigated area: Total,Irrigated area: Rice,Irrigated area: Wheat,Irrigated area: Maize,Irrigated area: Vegetables and fruits,Irrigated area: Others,Irrigation water-use intensity (WUI): Total,...,Rural domestic WU,Rural population,Rural domestic WUI,Rural livestock WU,Livestock population,Livestock WUI,Total water use,area,Perfecture,Province_n
0,C1,1965,1.229927,127.315625,59.834838,8.289719,0.443002,2.697220,56.050846,966.045314,...,0.014680,0.469140,85.727127,0.001804,110.158490,0.016373,1.261649,15372.895035,C1,Anhui
1,C1,1966,1.274452,129.968097,62.355186,7.819602,0.404165,2.741800,56.647345,980.588287,...,0.016790,0.516349,89.085128,0.001896,115.778437,0.016380,1.308981,15372.895035,C1,Anhui
2,C1,1967,1.482867,152.163380,68.736923,9.776923,0.489071,3.267321,69.893142,974.522696,...,0.018158,0.561759,88.556282,0.001993,121.665631,0.016385,1.519409,15372.895035,C1,Anhui
3,C1,1968,1.538731,156.787722,74.005815,10.511051,0.529418,3.443728,68.297710,981.410519,...,0.019248,0.605370,87.108619,0.002095,127.834367,0.016389,1.577010,15372.895035,C1,Anhui
4,C1,1969,1.642901,165.174460,83.210254,10.657419,0.569342,3.536641,67.200804,994.645761,...,0.020938,0.647184,88.636981,0.002201,134.298872,0.016391,1.683614,15372.895035,C1,Anhui


**Output digestion:**

---
**Analysis:**
这个数据级我们相对比较熟悉了，主要包括了：
- 农业灌溉面积（包括总的，和各种作物的面积）以及每个种植作物的单位面积耗水量（通过作物耗水量计算得出）
- 多种工业的总产值（GVA），已经是去通货膨胀之后的数据，可以直接用，还有每个产业的单位产出用水量，也是通过水量计算得出的
- 城市居民用水、城市人口
- 农村居民用水、农村人口
- 农村牲畜
- 城市服务业产出

## Merge source datasets

**Abstract:**
One-sentence description

**Description:**
In the following cell, I...


In [5]:
# 整合成 Panel 数据
agg_data = values_data.groupby(['Province_n', 'Year']).sum().reset_index().rename({'Province_n': 'Province'}, axis=1)
merged_data = pd.merge(how='left', left=agg_data, right=driver_data, on=['Province', 'Year'])

for col in merged_data:
    if "WU" in col:
        merged_data.drop(col, axis=1, inplace=True)
        print("Column {} dropped.".format(col))
        # 与WUI有关的列其实都不能用来预测
        
merged_data.head()

jinji = merged_data[merged_data['Province'].isin(['Hebei', 'Tianjin'])].groupby('Year').sum().reset_index()
jinji.loc[:, 'Province'] = 'Jinji'
jinji.shape

# 为每个单位制作一个独特的、数字的标签
province_index = {}
i = 1
for x in merged_data['Province']:
    if x not in province_index:
        province_index[x] = i
        i += 1
    
province_index['Jinji'] = 32  # 天津河北作为第32个地理单元
index_province = {province_index[province]: province for province in province_index.keys()}


merged_data = merged_data[~merged_data['Province'].isin(['Hebei', 'Tianjin'])].append(jinji)
merged_data['Index'] = merged_data['Province'].map(province_index)
merged_data = merged_data.reset_index(drop=True)


# 本研究只使用1975--2013年的数据，其它的不 care
mask = merged_data['Year'].isin(np.arange(1975, 2013))
merged_data = merged_data[mask]

print("\nShape after drop:")
merged_data.shape

Column Irrigation water-use intensity (WUI): Total dropped.
Column Irrigation water-use intensity (WUI): Rice dropped.
Column Irrigation water-use intensity (WUI): Wheat dropped.
Column Irrigation water-use intensity (WUI): Maize dropped.
Column Irrigation water-use intensity (WUI): Vegetables and fruits dropped.
Column Irrigation water-use intensity (WUI): Others dropped.
Column Industrial WUI: Total dropped.
Column Industrial WUI: Textile dropped.
Column Industrial WUI: Papermaking dropped.
Column Industrial WUI: Petrochemicals dropped.
Column Industrial WUI: Metallurgy dropped.
Column Industrial WUI: Mining dropped.
Column Industrial WUI: Food dropped.
Column Industrial WUI: Cements dropped.
Column Industrial WUI: Machinery dropped.
Column Industrial WUI: Electronics dropped.
Column Industrial WUI: Thermal electrivity dropped.
Column Industrial WUI: Others dropped.
Column Urban domestic WU dropped.
Column Urban domestic WUI dropped.
Column Urban service WU dropped.
Column Service WU

,Province,Year,IRR,Irrigated area: Total,Irrigated area: Rice,Irrigated area: Wheat,Irrigated area: Maize,Irrigated area: Vegetables and fruits,Irrigated area: Others,IND,...,RUR,Rural population,Livestock population,Total water use,area,PIRR,AIRR,WCI,Ratio of industrial water recycling,Ratio of industrial water evaporated
0,Anhui,1965,12.501807,1772.118366,430.460632,478.977784,43.664942,28.931495,790.083513,0.756417,...,0.403485,17.583737,1531.866330,13.872984,140116.916198,NaN,NaN,NaN,NaN,NaN
1,Anhui,1966,12.954392,1809.038381,448.592386,451.814526,39.836908,29.601534,839.193027,0.760892,...,0.469265,19.434819,1618.484472,14.411765,140116.916198,NaN,NaN,NaN,NaN,NaN
2,Anhui,1967,15.072861,2117.976644,494.503542,564.908028,48.205742,35.290717,975.068614,0.792135,...,0.491417,21.215645,1708.813710,16.590930,140116.916198,NaN,NaN,NaN,NaN,NaN
3,Anhui,1968,15.640705,2182.343303,532.408732,607.325742,52.182585,37.349555,953.076689,0.824855,...,0.545884,22.926217,1803.232978,17.259768,140116.916198,NaN,NaN,NaN,NaN,NaN
4,Anhui,1969,16.699555,2299.079112,598.626816,615.782835,56.117819,38.284127,990.267514,0.859135,...,0.573847,24.566534,1902.084965,18.390515,140116.916198,NaN,NaN,NaN,NaN,NaN


(49, 35)


Shape after drop:


/var/folders/m8/qpvn85g95rbgw5j_qb5g2pzh0000gn/T/ipykernel_50604/4083157166.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data[~merged_data['Province'].isin(['Hebei', 'Tianjin'])].append(jinji)


(1140, 36)

**Output digestion:**

---
**Analysis:**
1. 
2. 
3. 

## Export Data

**Abstract:**
One-sentence description

**Description:**
In the following cell, I...


In [6]:
# existed dataset.
pd.read_csv(os.path.join(ROOT, "data/processed/merged_data.csv")).shape

(1140, 37)

In [7]:
# 导出预处理之后的数据，用以在其它工作中分析
merged_data.to_csv(os.path.join(ROOT, "data/processed/merged_data.csv"))
merged_data.shape

(1140, 36)

**Output digestion:**

---
**Analysis:**
1. 
2. 
3. 

## GDP

**Abstract:**
One-sentence description

**Description:**
In the following cell, I...


In [18]:
gdp = pd.read_excel("../data/source/CRE_Gdp01.xlsx").groupby(["Sgnyea"])
gdp.head()

# gdp_1 = pd.DataFrame()
# gdp_2 = pd.DataFrame()
# gdp_3 = pd.DataFrame()
# for col in gdp.columns:
#     ch_pro = PROVINCES_CHN2ENG.get(col.split(':')[0])
#     industry = col.split(':')[-1]
#     if ch_pro:
#         if industry == '第一产业':
#             gdp_1[ch_pro] = gdp[col]
#         elif industry == '第二产业':
#             gdp_2[ch_pro] = gdp[col]
#         elif industry == '第三产业':
#             gdp_3[ch_pro] = gdp[col]
# gdp_1.shape

,Sgnyea,Prvcnm_id,Prvcnm,Gdp0101,Gdp0102,Gdp0103,Gdp0104,Gdp0105,Gdp0106,Gdp0107,...,Gdp0111,Gdp0112,Gdp0113,Gdp0114,Gdp0115,Gdp0116,Gdp0126,Gdp0127,Gdp0128,Gdp0131
0,年度标识,省份编码,省份名称,地区生产总值,地区生产总值－第一产业,地区生产总值－第二产业,地区生产总值－工业,地区生产总值－建筑业,地区生产总值－第三产业,地区生产总值－交通运输、仓储和邮政业,...,地区生产总值－房地产业,地区生产总值－其他服务业,第一产业占GDP比重(%),第二产业占GDP比重(%),第三产业占GDP比重(%),人均地区生产总值,地区生产总值指数（上年＝100）,地区生产总值指数－第一产业（上年＝100）,地区生产总值指数－第二产业（上年＝100）,地区生产总值指数－第三产业（上年＝100）
1,1952,142,中国,679.1,342.9,141.1,119.6,22,195.1,29,...,14,52,50.5,20.8,28.7,119,NaN,NaN,NaN,NaN
2,1952,340000,安徽省,22.88,17.18,2.27,1.77,0.5,3.43,0.7,...,NaN,NaN,75.0874,9.9213,14.9913,78,100,100,100,100
3,1952,330000,浙江省,24.53,16.28,2.78,2.29,0.49,5.47,0.98,...,NaN,NaN,66.3677,11.3331,22.2992,12,115.07,116.69,114.29,111.23
4,1952,220000,吉林省,16.55,9.19,4.54,4.19,0.35,2.82,0.78,...,NaN,NaN,55.5287,27.432,17.0393,153,100,100,100,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2059,2019,440000,广东省,107671.07,4351.26,43546.43,39398.46,4255.07,59773.38,3466.42,...,9223.62,24936.34,4,40.4,55.5,94172,106.2,104.1,104.7,107.5
2060,2019,310000,上海市,38155.32,103.88,10299.16,9670.68,716.16,27752.28,1650.44,...,3300.72,10624.53,0.3,27,72.7,157279,106,95,100.5,108.2
2061,2019,410000,河南省,54259.2,4635.4,23605.79,18413.21,5272.97,26018.01,2970.41,...,3419.58,11367.75,8.5,43.5,48,56387.84,107,102.3,107.5,107.4
2062,2019,220000,吉林省,11726.82,1287.32,4134.82,3347.82,808.63,6304.68,574.4,...,778.83,3094.12,11,35.2,53.8,43475,103,102.5,102.6,103.3


In [9]:
gdp_2.shape

(71, 30)

In [10]:
gdp_3.shape

(71, 31)

In [11]:
gdp_2.columns
gdp_1.columns

Index(['Beijing', 'Tianjin', 'Hebei', 'Shaanxi', 'Neimeng', 'Liaoning',
       'Jilin', 'Heilongjiang', 'Shanghai', 'Jiangsu', 'Zhejiang', 'Anhui',
       'Fujian', 'Jiangxi', 'Shandong', 'Henan', 'Hubei', 'Guangdong',
       'Guangxi', 'Hainan', 'Chongqing', 'Sichuan', 'Guizhou', 'Yunnan',
       'Tibet', 'Shanxi', 'Gansu', 'Qinghai', 'Ningxia', 'Xinjiang'],
      dtype='object')

Index(['Beijing', 'Tianjin', 'Hebei', 'Shaanxi', 'Neimeng', 'Liaoning',
       'Jilin', 'Heilongjiang', 'Shanghai', 'Jiangsu', 'Zhejiang', 'Anhui',
       'Fujian', 'Jiangxi', 'Shandong', 'Henan', 'Hubei', 'Hunan', 'Guangdong',
       'Guangxi', 'Hainan', 'Chongqing', 'Sichuan', 'Guizhou', 'Yunnan',
       'Tibet', 'Shanxi', 'Gansu', 'Qinghai', 'Ningxia', 'Xinjiang'],
      dtype='object')

In [12]:
gdp

,北京:GDP:第一产业,北京:GDP:第二产业,北京:GDP:第三产业,天津:GDP:第一产业,天津:GDP:第二产业,天津:GDP:第三产业,河北:GDP:第一产业,河北:GDP:第二产业,河北:GDP:第三产业,山西:GDP:第一产业,...,新疆:GDP:第三产业,兵团:GDP:第一产业,兵团:GDP:第二产业,兵团:GDP:第三产业,珠三角:GDP:第一产业,珠三角:GDP:第二产业,珠三角:GDP:第三产业,长三角:GDP:第一产业,长三角:GDP:第二产业:合计,长三角:GDP:第三产业:合计
Year,,,,,,,,,,,,,,,,,,,,,
1949,0.64,1.02,1.11,0.94,1.48,1.65,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1950,1.07,1.71,1.85,1.22,3.59,2.74,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1951,1.76,2.83,3.07,1.62,6.13,3.7,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1952,1.75,3.05,3.08,1.85,6.31,4.64,25.23,7.61,7.65,9.38,...,1.05,0,0,0,0,0,0,0,0,0
1953,3.37,7.85,8.06,1.76,8.49,7.33,24.1,8.38,10,11.05,...,1.13,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,140.21,4542.64,18331.74,208.82,7704.22,8625.15,3439.45,14386.87,11979.79,783.16,...,4169.32,4280411,8838760,6229951,1073.87,28135.99,34171.99,8401.18,71948.59,81491.04
2016,129.79,4944.44,20594.9,220.22,7571.35,10093.82,3492.81,15256.93,13320.71,784.78,...,4353.72,4678739,9655779,7008789,1153.92,29692.8,38223.54,8724.42,75810.64,92405.98
2017,120.42,5326.76,22567.76,168.96,7593.59,10786.64,3129.98,15846.21,15040.13,719.16,...,4999.23,5063290,10265022,8062416,1181.53,31542.82,42985.8,8703.62,83055.88,103560.7


In [13]:
merged_data.Province.unique()

array(['Anhui', 'Beijing', 'Chongqing', 'Fujian', 'Gansu', 'Guangdong',
       'Guangxi', 'Guizhou', 'Hainan', 'Heilongjiang', 'Henan', 'Hubei',
       'Hunan', 'Jiangsu', 'Jiangxi', 'Jilin', 'Liaoning', 'Neimeng',
       'Ningxia', 'Qinghai', 'Shaanxi', 'Shandong', 'Shanghai', 'Shanxi',
       'Sichuan', 'Tibet', 'Xinjiang', 'Yunan', 'Zhejiang', 'Jinji'],
      dtype=object)

In [14]:
len(merged_data.Province.unique())

30